#Setup

##Imports

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

import sys
sys.path.append('/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch')

import os
from image import *
from model import CSRNet
import glob
import torch

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##Variables
variables à changer en fonction de l'output espéré

In [22]:
#parameters to change
INPUT_MODEL_NAME = 'PartAmodel_best.pth.tar'
OUTPUT_MODEL_NAME = 'PartAmodel_best_1024' #Wil be a directory with the json and the bins, usable with js
Input_img_size = (768,1024)
use_gpu = True

#paths

In [34]:
root = '/content/gdrive/MyDrive/TFE_crowd_counting'
dataset_path = os.path.join(root,'Dataset')
images_path = os.path.join(dataset_path,'images')

path_sets = [images_path]  #All images at the same place
scripts_path = os.path.join(root,'CSRNet-pytorch')
models_path = os.path.join(scripts_path,'models')
input_model_path = os.path.join(models_path,INPUT_MODEL_NAME)
onnx_model_path = os.path.join(models_path, OUTPUT_MODEL_NAME+'.onnx')
tf_model_path = os.path.join(models_path, OUTPUT_MODEL_NAME+'.pb')
output_path = os.path.join(models_path, OUTPUT_MODEL_NAME)


img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [19]:
#return the loaded model located at model_path (or the basic shangai partAmodel if no path is given ) 
#using gpu or using cpu (if the use_gpu parameter is set to False)
def load_model(model_path = '/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/model_best.pth.tar', use_gpu = use_gpu):
  from torchvision import datasets, transforms
  transform=transforms.Compose([
                        transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
                    ])
  model = CSRNet()
  if use_gpu:
    model = model.cuda()
  else:
    model = model.cpu()
  checkpoint = torch.load(model_path)
  model.load_state_dict(checkpoint['state_dict'])
  return model

#Conversion

In [ ]:
#Step 1: convert to onnx
model = load_model(input_model_path)
input_names = [ "image" ]
output_names = [ "output1" ]
if use_gpu:
  dummy_input = torch.randn(1 ,3, Input_img_size[0], Input_img_size[1], device='cuda')
else:
  dummy_input = torch.randn(1 ,3, Input_img_size[0], Input_img_size[1])
torch.onnx.export(model, dummy_input, onnx_model_path, operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)

In [ ]:
#Step 2: export to tensorflow
!pip install git+https://github.com/onnx/onnx-tensorflow.git #May be commented if the package is already installed
!onnx-tf convert -i {onnx_model_path} -o {tf_model_path}

In [ ]:
#Step 3: export to tensorflow.js
!pip install tensorflowjs #May be commented if the package is already installed

In [35]:
#Faut copier coller les 2 paths et les donner en argument au programme à la place de ceux qui sont là... Je sais pas pourquoi avec tensorflow on peut pas donner de variable...
print(tf_model_path)
print(output_path)

/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/PartAmodel_best_1024.pb
/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/PartAmodel_best_1024


In [36]:
!tensorflowjs_converter --input_format='tf_saved_model' '/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/PartAmodel_best_1024.pb' '/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/PartAmodel_best_1024'

2022-02-23 11:29:36.374138: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Writing weight file /content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/models/PartAmodel_best_1024/model.json...
